In [1]:
!pip install -q transformers

/bin/bash: /home/anwer/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [1]:
import torch
from transformers import BertModel, BertTokenizer
import re
import os
import requests
from tqdm.auto import tqdm

In [2]:
tokenizer = BertTokenizer.from_pretrained('Rostlab/prot_bert_bfd', do_lower_case=False )

In [3]:
model = BertModel.from_pretrained("Rostlab/prot_bert_bfd")

Some weights of the model checkpoint at Rostlab/prot_bert_bfd were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

In [5]:
model = model.to(device)
model = model.eval()

In [14]:

sequences_Example = ["A E T C Z A O","S K T Z P"]# sequences_Example = ["A E T C Z A O"]
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]
sequences_Example

['A E T C X A X', 'S K T X P']

In [15]:
ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, pad_to_max_length=True)

In [16]:
ids

{'input_ids': [[2, 6, 9, 15, 23, 25, 6, 25, 3], [2, 10, 12, 15, 25, 16, 3, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 0, 0]]}

In [17]:
input_ids = torch.tensor(ids['input_ids']).to(device)
attention_mask = torch.tensor(ids['attention_mask']).to(device)

In [18]:
with torch.no_grad():
    embedding = model(input_ids=input_ids,attention_mask=attention_mask)[0]

In [54]:

features = [] 
for seq_num in range(len(embedding)):
    seq_len = (attention_mask[seq_num] == 1).sum()
    seq_emd = embedding[seq_nu][1:seq_len-1]
    seq_emd = np.mean(seq_emd,axis = 0)
    features.append(seq_emd)

In [55]:
features


[array([ 0.07877477, -0.07100276, -0.03651459, ...,  0.02056901,
         0.05276874,  0.07941906], dtype=float32),
 array([ 0.00065715, -0.17282128, -0.07624378, ..., -0.04359809,
        -0.02556973, -0.04013642], dtype=float32)]

In [28]:
llfeatures[0]

array([[ 0.05550989, -0.10461241, -0.03254269, ...,  0.05091707,
         0.04319475,  0.10180821],
       [ 0.13895592, -0.04658469,  0.02193582, ...,  0.0694277 ,
         0.14762917,  0.06504028],
       [ 0.1461069 , -0.08092842, -0.12500228, ..., -0.03651199,
         0.02485694,  0.07977536],
       ...,
       [ 0.02349972, -0.01549867, -0.05685189, ..., -0.01342195,
         0.01704335,  0.06431139],
       [ 0.08130013, -0.10929574, -0.03022898, ...,  0.08717727,
         0.0206151 ,  0.05156785],
       [ 0.06197315, -0.06417973, -0.02039701, ..., -0.02796507,
         0.08840097,  0.07532778]], dtype=float32)

In [44]:
l = 0
for i in range(len(features[0])):
    print(features[0][i][0])
    l =+ features[0][i][0]/

0.055509888
0.13895592
0.1461069
0.04407769
0.023499716
0.08130013
0.06197315


In [48]:
import numpy as np
np.sum(features[0],axis = 0)

array([ 0.55142343, -0.4970193 , -0.25560215, ...,  0.14398307,
        0.3693812 ,  0.5559334 ], dtype=float32)

In [51]:
import numpy as np
np.sum(features[1],axis = 0)

array([ 0.00328575, -0.8641064 , -0.3812189 , ..., -0.21799044,
       -0.12784865, -0.20068212], dtype=float32)

In [49]:
0.055509888+0.13895592+0.1461069+0.04407769+0.023499716+0.08130013+0.06197315

0.5514233940000001

In [57]:
0.5514233940000001/7

0.07877477057142858